# BoxBunny Unit Tests
Essential unit tests for validating system components.

## Setup Environment
Run this first to configure the shell environment.

In [ ]:
%%bash
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH
export SETUPTOOLS_USE_DISTUTILS=stdlib
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash 2>/dev/null || true
SH
echo "Environment ready"

## Build Workspace
**Critical:** Run this to build `boxbunny_msgs` and other packages.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
colcon build --symlink-install
source install/setup.bash

## 1. GUI Design Test
Test the main robot GUI appearance and layout.
**Expectation:** The main BoxBunny GUI opens. You can navigate menus to verify design.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/src/boxbunny_gui/boxbunny_gui
python gui_main.py

## 2. IMU Calibration Test
Test the IMU sensors and punch calibration interface.
**Expectation:** The IMU Calibration GUI opens. You can see accelerometer/gyroscope data.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
# Launches IMU node + Classifier + Calibration GUI
ros2 launch boxbunny_bringup imu_only.launch.py

## 3. Reaction Drill Test (Unified Launch)
Verify the full reaction drill workflow with a single command.
**Expectation:** 
1. **Everything starts automatically** (Drill Manager, AI, GUI).
2. GUI defaults to **Action / Pose Model** mode.
3. You should see video + skeleton overlay immediately.
4. Click **START REACTION DRILL**.

In [58]:
%%bash
source /tmp/boxbunny_env.sh

# AGGRESSIVELY kill conflicting camera nodes
pkill -f realsense2_camera_node || true
pkill -f live_infer_rgbd.py || true
# Kill any ros launch processes that might restart nodes
pkill -f "ros2 launch boxbunny_bringup vision_only.launch.py" || true
sleep 2

cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws

# Rebuild drills package to install new launch file
colcon build --packages-select boxbunny_drills --symlink-install > /dev/null
source install/setup.bash

# Launch Unified Reaction Drill
ros2 launch boxbunny_drills reaction_drill.launch.py

[0.867s] WARNING:colcon.colcon_core.package_selection:Some selected packages are already built in one or more underlay workspaces:
	'boxbunny_drills' is in: /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/boxbunny_drills
If a package in a merged underlay workspace is overridden and it installs headers, then all packages in the overlay must sort their include directories by workspace order. Failure to do so may result in build failures or undefined behavior at run time.
If the overridden package is used by another package in any underlay, then the overriding package in the overlay must be API and ABI compatible or undefined behavior at run time may occur.

If you understand the risks and want to override a package anyways, add the following to the command line:
	--allow-overriding boxbunny_drills

This may be promoted to an error in a future release of colcon-override-check.


[INFO] [launch]: All log files can be found below /home/boxbunny/.ros/log/2026-01-29-21-41-27-656472-ubuntu-69503
[INFO] [launch]: Default logging verbosity is set to INFO
[INFO] [reaction_drill_manager-1]: process started with pid [69504]
[INFO] [python3-2]: process started with pid [69506]
[INFO] [python3-3]: process started with pid [69508]
[reaction_drill_manager-1] [INFO] [1769694088.042922620] [reaction_drill_manager]: Reaction drill manager ready
[python3-3] [INFO] [1769694089.020552674] [vision_debug_gui]: Vision GUI Node initialized
[python3-2] Set torch hub dir to: /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction
[python3-2] Loading Pose model...
[python3-2] Starting camera...
[python3-2] Camera started successfully.
[python3-2] Headless Inference Running. Press Ctrl+C to stop.
[reaction_drill_manager-1] Traceback (most recent call last):
[reaction_drill_manager-1]   File "/home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/boxbunn

TypeError: %d format: a real number is required, not NoneType

## 4. Punch Detection Test (Unified Launch)
Test Vision & IMU based punch detection with a single command.
**Expectation:** 
1. Starts Vision (Color/Glove) System + IMU System + Debug GUI.
2. GUI Defaults to **Color Tracking**.
3. Monitor 'Punch Log' for events and verify glove tracking video.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
pkill -f realsense2_camera_node || true
pkill -f live_infer_rgbd.py || true

cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws

# Ensure updated launch files are available
source install/setup.bash

# Launch Unified Punch Detection
ros2 launch boxbunny_drills punch_detection.launch.py

## 5. Height Measurement Check
Verify that user height is estimated from Pose data.
**Expectation:**
1. Launch the Reaction Drill (Step 3).
2. Stand in front of the camera (ensure full body/legs visible preferably).
3. In a separate terminal, echo the height topic:
   `ros2 topic echo /player_height`
4. You should see float values (approx 1.7-1.9m) streaming.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
echo "Listening for height updates (Ctrl+C to stop)..."
ros2 topic echo /player_height